In [1]:
import sys
sys.path.insert(0, "../../code")

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot as plt
import ariadne as a

from datasets import *
from api import *
from temporary_generator import *

reading TSV: ../../data/raw/gcp/datasets/StandingSettlementInstructions/SSIPLUS_V3_MONTHLY_FULL_20200828.txt


../../code/api.py:257: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  num_payments = _num_payments,


reading TSV: ../../data/raw/gcp/datasets/StandingSettlementInstructions/BANKDIRECTORYPLUS_V3_FULL_20200828.txt


../../code/api.py:257: DtypeWarning: Columns (10,11,12,13,14,16,17,20,21,22,23,24,25,27,30,31,32,33,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  num_payments = _num_payments,


In [3]:
def assert_equals(expected, actual):
    if (expected == actual):
        return True
    else:
        raise ValueError(
            "Not equal!\n\nexpected:\n%s\n\nactual:\n%s\n" % (str(expected), str(actual))
        )

### Resolvers

In [4]:
r = DatasetsResolvers()

In [5]:
assert_equals(
    Party(
        bdp = "BD_CLIENT_Z2",
        bic = "BDABLABX840",
        name = "Z2 Corp.",
        countryCode = "SG",
        countryName = "United Kingdom",
        city = "London",
    ),
    r._client_party(),
)

True

In [6]:
assert_equals(
    [
        Party(bdp='BD0000001QJP', bic='RZOOAT2L455', name='Raiffeisenbank Region Schaerding eGen', countryCode='AT', countryName='AUSTRIA', city='Schaerding'),
        Party(bdp='BD000000RYHO', bic='BINHCN2NXXX', name='NINGBO COMMERCE BANK', countryCode='CN', countryName='CHINA', city='NINGBO'),
        Party(bdp='BD0000001H96', bic='NOLADE21STK', name='KREISSPARKASSE STADE', countryCode='DE', countryName='GERMANY', city='STADE'),
        Party(bdp='BD0000000CD2', bic='BNPADZALXXX', name='BNP PARIBAS EL DJAZAIR', countryCode='DZ', countryName='ALGERIA', city='ALGIERS'),
        Party(bdp='BD000000EVYD', bic='HNBKTWTP601', name='HUA NAN COMMERCIAL BANK, LTD.', countryCode='TW', countryName='TAIWAN', city='CHAI YI'),
    ],
    r._client_banks()
)

True

In [7]:
assert_equals(
    [
        Destination(
            party=Party(bdp='BD00000006O3', bic='BBACLBBXXXX', name='BBAC SAL', countryCode='LB', countryName='LEBANON', city='BEIRUT'),
            accounts=['0001736803002', 'USD 0001 796306-001', '0001 784202 001'],
        ),
        Destination(
            party=Party(bdp='BD0000000HUQ', bic='BSHROMRUXXX', name='SOHAR INTERNATIONAL BANK S.A.O.G', countryCode='OM', countryName='OMAN', city='MUSCAT'),
            accounts=['001020088035', '001020088026', '001020087896', '001020097024', '1020038836', '001020068518', '001020035404', '001020058867', '001020074120', '1020058867', '001020092560', '001020093392', '001020076775'],
        ),
    ],
    r._client_destinations(),
)

True

In [8]:
assert_equals(
    [
        'ANYY',
        'WHLS',
        'COPA',
        'TREA',
        'FOEX',
        'NDLF',
        'OPTI',
        'DERI',
        'MMKT',
        'LOAN',
        'SECU',
        'COLL',
        'GUAR',
        'COMM',
        'LETT',
        'TFIN',
        'DOCC',
        'CASH',
    ],
    r._asset_categories(),
)

True

In [9]:
assert_equals(
    StaticData(
        origins = [
            Party(bdp='BD0000001QJP', bic='RZOOAT2L455', name='Raiffeisenbank Region Schaerding eGen', countryCode='AT', countryName='AUSTRIA', city='Schaerding'),
            Party(bdp='BD000000RYHO', bic='BINHCN2NXXX', name='NINGBO COMMERCE BANK', countryCode='CN', countryName='CHINA', city='NINGBO'),
            Party(bdp='BD0000001H96', bic='NOLADE21STK', name='KREISSPARKASSE STADE', countryCode='DE', countryName='GERMANY', city='STADE'),
            Party(bdp='BD0000000CD2', bic='BNPADZALXXX', name='BNP PARIBAS EL DJAZAIR', countryCode='DZ', countryName='ALGERIA', city='ALGIERS'),
            Party(bdp='BD000000EVYD', bic='HNBKTWTP601', name='HUA NAN COMMERCIAL BANK, LTD.', countryCode='TW', countryName='TAIWAN', city='CHAI YI'),
        ],
        destinations = [
            Destination(
                party=Party(bdp='BD00000006O3', bic='BBACLBBXXXX', name='BBAC SAL', countryCode='LB', countryName='LEBANON', city='BEIRUT'),
                accounts=['0001736803002', 'USD 0001 796306-001', '0001 784202 001'],
            ),
            Destination(
                party=Party(bdp='BD0000000HUQ', bic='BSHROMRUXXX', name='SOHAR INTERNATIONAL BANK S.A.O.G', countryCode='OM', countryName='OMAN', city='MUSCAT'),
                accounts=['001020088035', '001020088026', '001020087896', '001020097024', '1020038836', '001020068518', '001020035404', '001020058867', '001020074120', '1020058867', '001020092560', '001020093392', '001020076775'],
            ),
        ],
        assetCategories=['ANYY', 'WHLS', 'COPA', 'TREA', 'FOEX', 'NDLF', 'OPTI', 'DERI', 'MMKT', 'LOAN', 'SECU', 'COLL', 'GUAR', 'COMM', 'LETT', 'TFIN', 'DOCC', 'CASH'],
    ),
    r.staticData(),
)

True

In [10]:
assert_equals(
    [
        Payment(originBic='BD000000RYHO', destinationBic='BD0000000HUQ', assetCategory='DERI', currency='CHF', amount=2128525),
        Payment(originBic='BD0000001QJP', destinationBic='BD0000000HUQ', assetCategory='GUAR', currency='NZD', amount=343442),
        Payment(originBic='BD000000EVYD', destinationBic='BD00000006O3', assetCategory='LOAN', currency='EUR', amount=1374073),
        Payment(originBic='BD0000001QJP', destinationBic='BD0000000HUQ', assetCategory='DOCC', currency='JPY', amount=350662),
        Payment(originBic='BD0000001H96', destinationBic='BD0000000HUQ', assetCategory='COMM', currency='NZD', amount=652946),
        Payment(originBic='BD0000001H96', destinationBic='BD00000005G6', assetCategory='FOEX', currency='GBP', amount=1216130),
        Payment(originBic='BD000000EVYD', destinationBic='BD00000005G6', assetCategory='COPA', currency='EUR', amount=780335),
        Payment(originBic='BD000000EVYD', destinationBic='BD00000006O3', assetCategory='TREA', currency='CHF', amount=779841),
        Payment(originBic='BD000000EVYD', destinationBic='BD0000000HUQ', assetCategory='LOAN', currency='CHF', amount=978510),
        Payment(originBic='BD000000RYHO', destinationBic='BD00000005G6', assetCategory='SECU', currency='NZD', amount=1459874),
    ],
    r.payments(),
)

True

In [13]:
g = TemporaryGenerator()
pmt = g.generate_payment()
assert_equals(
    Route(
        originalPayment=Payment(originBic='BD_CLIENT_Z2', destinationBic='BD00000005G6', assetCategory='DERI', currency='AUD', amount=3612529),
        hops=[
            Hop(source='BD_CLIENT_Z2', target='BD0000000CD2', 
                payment=Payment(originBic='BD_CLIENT_Z2', destinationBic='BD0000000CD2', assetCategory='DERI', currency='USD', amount=8677240), 
                fxRate=2.40234609000285, timeTakenMinutes=693, crossBorder=True, charge=543), 
            Hop(source='BD0000000CD2', target='BD0000001Y1B', 
                payment=Payment(originBic='BD0000000CD2', destinationBic='BD0000001Y1B', assetCategory='DERI', currency='CHF', amount=3514384), 
                fxRate=0.9731305537256456, timeTakenMinutes=1897, crossBorder=True, charge=1108), 
            Hop(source='BD0000001Y1B', target='BD0000000L5T', 
                payment=Payment(originBic='BD0000001Y1B', destinationBic='BD0000000L5T', assetCategory='DERI', currency='CHF', amount=6782627), 
                fxRate=1.8776255361186642, timeTakenMinutes=1697, crossBorder=True, charge=186), 
            Hop(source='BD0000000L5T', target='BD00000005G6', 
                payment=Payment(originBic='BD0000000L5T', destinationBic='BD00000005G6', assetCategory='DERI', currency='GBP', amount=8235835), 
                fxRate=2.2804016207635724, timeTakenMinutes=325, crossBorder=True, charge=957)
        ], risk='LO', totalFee=2794, totalTimeMinutes=4612, success = True
    ),
    g.generate_route(
        pmt,
        r._d.with_client_ssi_nx,
        pmt.originBic,
        pmt.destinationBic
    )
)

True

### Generator

In [14]:
rnd = random.Random(100500)

In [15]:
g = TemporaryGenerator()

In [16]:
pmt = g.generate_payment()

In [18]:
payments

[Payment(originBic='BD000000RYHO', destinationBic='BD0000000HUQ', assetCategory='DERI', currency='CHF', amount=2128525),
 Payment(originBic='BD0000001QJP', destinationBic='BD0000000HUQ', assetCategory='GUAR', currency='NZD', amount=343442),
 Payment(originBic='BD000000EVYD', destinationBic='BD00000006O3', assetCategory='LOAN', currency='EUR', amount=1374073),
 Payment(originBic='BD0000001QJP', destinationBic='BD0000000HUQ', assetCategory='DOCC', currency='JPY', amount=350662),
 Payment(originBic='BD0000001H96', destinationBic='BD0000000HUQ', assetCategory='COMM', currency='NZD', amount=652946),
 Payment(originBic='BD0000001H96', destinationBic='BD00000005G6', assetCategory='FOEX', currency='GBP', amount=1216130),
 Payment(originBic='BD000000EVYD', destinationBic='BD00000005G6', assetCategory='COPA', currency='EUR', amount=780335),
 Payment(originBic='BD000000EVYD', destinationBic='BD00000006O3', assetCategory='TREA', currency='CHF', amount=779841),
 Payment(originBic='BD000000EVYD', de

In [66]:
payments = r._d.payments_history
routes = r._d.routes_history

In [67]:
total_volume = sum([payment.amount for payment in payments])
average_time_minutes = sum([route.totalTimeMinutes for route in routes]) / len(routes)
pct_failures = sum([1 for route in routes if not route.success]) / len(routes)

In [68]:
total_volume, average_time_minutes, pct_failures

(10064338, 2598.0, 0.1)

In [64]:
source_cities = [ party.city for party in 
    [ r._parties_by_bdps([payment.originBic])[0] for payment in payments ]
]
target_cities = [ party.city for party in 
    [ r._parties_by_bdps([payment.destinationBic])[0] for payment in payments ]
]
weights = [ route.originalPayment.amount for route in routes ]

pd.DataFrame.from_records(
    zip(source_cities, target_cities, weights),
    columns = [ 'source_city', 'target_city', 'weight' ]
).groupby([ 'source_city', 'target_city' ]).sum().reset_index().apply(
    lambda x: MapEdge(x['source_city'], x['target_city'], x['weight']),
    axis = 1
).values

array([MapEdge(sourceCity='CHAI YI', targetCity='BEIRUT', weight=2153914),
       MapEdge(sourceCity='CHAI YI', targetCity='MANAMA', weight=780335),
       MapEdge(sourceCity='CHAI YI', targetCity='MUSCAT', weight=978510),
       MapEdge(sourceCity='NINGBO', targetCity='MANAMA', weight=1459874),
       MapEdge(sourceCity='NINGBO', targetCity='MUSCAT', weight=2128525),
       MapEdge(sourceCity='STADE', targetCity='MANAMA', weight=1216130),
       MapEdge(sourceCity='STADE', targetCity='MUSCAT', weight=652946),
       MapEdge(sourceCity='Schaerding', targetCity='MUSCAT', weight=694104)], dtype=object)

In [69]:
source_bdps = [ party.bdp for party in 
    [ r._parties_by_bdps([payment.originBic])[0] for payment in payments ]
]
target_bdps = [ party.bdp for party in 
    [ r._parties_by_bdps([payment.destinationBic])[0] for payment in payments ]
]

In [70]:
pd.DataFrame.from_records(
    zip(source_bdps, target_bdps),
    columns = [ 'source_bdp', 'target_bdp' ]
).drop_duplicates()

,source_bdp,target_bdp
0,BD000000RYHO,BD0000000HUQ
1,BD0000001QJP,BD0000000HUQ
2,BD000000EVYD,BD00000006O3
4,BD0000001H96,BD0000000HUQ
5,BD0000001H96,BD00000005G6
6,BD000000EVYD,BD00000005G6
8,BD000000EVYD,BD0000000HUQ
9,BD000000RYHO,BD00000005G6


,source_bdp,target_bdp,total_amount
0,BD0000001H96,BD00000005G6,1216130
1,BD0000001H96,BD0000000HUQ,652946
2,BD0000001QJP,BD0000000HUQ,694104
3,BD000000EVYD,BD00000005G6,780335
4,BD000000EVYD,BD00000006O3,2153914
5,BD000000EVYD,BD0000000HUQ,978510
6,BD000000RYHO,BD00000005G6,1459874
7,BD000000RYHO,BD0000000HUQ,2128525


In [106]:
opportunities = pd.DataFrame.from_records(
    [ (route.originalPayment.originBic, route.originalPayment.destinationBic, 
       route.totalTimeMinutes, route.success
      ) for route in routes ],
    columns = [ 'source_bdp', 'target_bdp', 'time_minutes', 'success' ]
).assign(
    failure = lambda x: np.where(x['success'], 0.0, 1.0)
).groupby([ 'source_bdp', 'target_bdp' ]).agg({
    'time_minutes' : [('average_time_minutes', 'mean')],
    'failure' : [
         ('total_failures', 'sum'),
         ('count_failures', 'count')
    ],
}).reset_index()
opportunities.columns = [
    'source_bdp', 'target_bdp', 'average_time_minutes', 'total_failures', 'count_failures'
]
opportunities = tmp.assign(
    pct_failures = lambda x: x['total_failures'] / x['count_failures'] * 100
)
opportunities.merge(
    pd.DataFrame.from_records(
        [ (payment.originBic, payment.destinationBic, payment.amount) for payment in payments ],
        columns = [ 'source_bdp', 'target_bdp', 'total_amount' ]
    ).groupby([ 'source_bdp', 'target_bdp' ]).sum().reset_index(),
    on = ('source_bdp', 'target_bdp')
)[[
    'source_bdp', 'target_bdp', 'total_amount', 'average_time_minutes', 'pct_failures'
]].apply(
    lambda x: Opportunity(
        r._parties_by_bdps([x['source_bdp']])[0],
        r._parties_by_bdps([x['target_bdp']])[0],
        Summary(
            x['total_amount'],
            x['average_time_minutes'],
            x['pct_failures'],
        )
    ),
    axis = 1
).values

array([ Opportunity(source=Party(bdp='BD0000001H96', bic='NOLADE21STK', name='KREISSPARKASSE STADE', countryCode='DE', countryName='GERMANY', city='STADE'), target=Party(bdp='BD00000005G6', bic='BAHLBHBMXXX', name='BANK AL HABIB LIMITED', countryCode='BH', countryName='BAHRAIN', city='MANAMA'), summary=Summary(totalVolume=1216130, averageTimeMinutes=1948.0, pctFailures=0.0)),
       Opportunity(source=Party(bdp='BD0000001H96', bic='NOLADE21STK', name='KREISSPARKASSE STADE', countryCode='DE', countryName='GERMANY', city='STADE'), target=Party(bdp='BD0000000HUQ', bic='BSHROMRUXXX', name='SOHAR INTERNATIONAL BANK S.A.O.G', countryCode='OM', countryName='OMAN', city='MUSCAT'), summary=Summary(totalVolume=652946, averageTimeMinutes=4056.0, pctFailures=100.0)),
       Opportunity(source=Party(bdp='BD0000001QJP', bic='RZOOAT2L455', name='Raiffeisenbank Region Schaerding eGen', countryCode='AT', countryName='AUSTRIA', city='Schaerding'), target=Party(bdp='BD0000000HUQ', bic='BSHROMRUXXX', name

In [ ]:
total_volume = sum([payment.amount for payment in payments])
average_time_minutes = sum([route.totalTimeMinutes for route in routes]) / len(routes)
pct_failures = sum([1 for route in routes if not route.success]) / len(routes)

In [109]:
[1, 2][0:3]

[1, 2]

In [116]:
def log(*args):
    print(*list(map(str, args)))



In [117]:
log(1, 2, 3)

1 2 3


In [119]:
{ i : 'a' for i in range(3) } 

{0: 'a', 1: 'a', 2: 'a'}